In [1]:
import yaml
from pathlib import Path
import os
import subprocess

In [2]:
root = Path('~/FCA_Fetal_Skin_priv').expanduser()
os.chdir(root)
!pwd

/home/jovyan/FCA_Fetal_Skin_priv


List of packages extracted from "~/FCA_Fetal_Skin_priv/code/notebooks/figures/fig2D.ipynb" as a starting reference for creating the base conda env (FCA_base) for the project.

In [3]:
def load_env(yml_file):
    """Loads the contents of a conda environment into a dictionary.
    --------------------------------------------------------------------
    Parameters:
    
    - yml_file: Path to the query environment's .yml file. (str)
    ----------
    outputs:
    
    1. env : {package_1 : version, ... , package_n : version}
    2. channels : [list, of, channels]
    """
    
    from collections import defaultdict
    
    assert type(yml_file) == str, "yml_file MUST be of type str!"
    
    env = defaultdict(list)
    channels = []
    
    with Path(yml_file).open(mode='r') as FCA_pakgs:
        package_list = yaml.load(FCA_pakgs, Loader=yaml.FullLoader)
        for item, doc in package_list.items():
            if 'dependencies' in item:
                for packg in doc:
                    try:
                        pack = '='.join(packg.split('==')).split('=')
                        try:
                            env['package'].append(pack[0])
                            env['version'].append( pack[1])
                            env['build'].append(pack[2])
                        except IndexError:
                            env['version'].append('unspecified')
                            env['build'].append('unspecified')
                    except AttributeError:
                        if 'pip' in packg:
                            for pip in packg['pip']:
                                pack = '='.join(pip.split('==')).split('=')
                                if pack[0] == pack[-1]:
                                    env['package'].append(pack[0])
                                    env['version'].append('unspecified')
                                    env['build'].append('unspecified')
                                else:
                                    env['package'].append(f'{pack[0]} **pip**')
                                    env['version'].append(pack[1])
                                    env['build'].append('unspecified')
                        else:
                            print(f'{Fore.RED}{packg} includes unrecognised sources of installation. Skipping the rest of the .yml file!{Style.RESET_ALL}')
                            break
            elif 'channels' in item:
                for ch in doc:
                    channels.append(ch)
                
    return env, channels

In [4]:
test_env, test_channel = load_env('logs/FCA_base.yml')

In [5]:
test_env

defaultdict(list,
            {'package': ['_libgcc_mutex',
              '_openmp_mutex',
              'adjusttext',
              'airr',
              'alsa-lib',
              'anndata',
              'arpack',
              'asttokens',
              'attr',
              'attrs',
              'backcall',
              'backports',
              'backports.functools_lru_cache',
              'backports.zoneinfo',
              'blas',
              'blosc',
              'brotli',
              'brotli-bin',
              'bzip2',
              'c-ares',
              'ca-certificates',
              'cellrank',
              'certifi',
              'click',
              'colorama',
              'contourpy',
              'curl',
              'cycler',
              'dbus',
              'debugpy',
              'decorator',
              'docrep',
              'dunamai',
              'entrypoints',
              'executing',
              'expat',
              'fa2',
   

In [6]:
class PackageNotFound(Exception):
    """Raised when the target packages are not found"""
    pass


def crossref_2_envs(path_to_env1, path_to_env2, packages=None, diff_only=False, compare_build=False):
    """Takes .yml file from two envs and cross-references the packages
    and version, if check_version is set to True."""
    
    from collections import OrderedDict
    from colorama import Fore, Style
    from math import floor, ceil
    
    assert type(path_to_env1) == str, "path_to_env1 must be a string"
    assert type(path_to_env2) == str, "path_to_env2 must be a string"
    if type(packages) == list:
        assert all(type(pckg) == str for pckg in packages), "All specified packages in packages must of type string!"
    
    env1, channels1 = load_env(path_to_env1)
    env2, channels2 = load_env(path_to_env2)
    if compare_build:
        env1 = {p:''.join([env1['version'][i], ''.join([' ' for i in range(16-len(env1['version'][i]))]), env1['build'][i]]) for i, p in enumerate(env1['package'])}
        env2 = {p:''.join([env2['version'][i], ''.join([' ' for i in range(16-len(env2['version'][i]))]), env2['build'][i]]) for i, p in enumerate(env2['package'])}
    else:
        env1 = {p:env1['version'][i] for i, p in enumerate(env1['package'])}
        env2 = {p:env2['version'][i] for i, p in enumerate(env2['package'])}

    env1_name = path_to_env1.split('/')[-1].split('.')[0]
    env2_name = path_to_env2.split('/')[-1].split('.')[0]
    crossref_diff = dict()
    for p in env1:
        if p not in env2.keys():
            crossref_diff[p] = 'env1'
        else:
            crossref_diff[p] = 'both'
    for p in env2:
        if p not in env1.keys():
            crossref_diff[p] = 'env2'
        else:
            continue
    crossref_diff = OrderedDict(sorted(crossref_diff.items()))
    if packages:
        if type(packages) == list:
            packages = [pckg.lower() for pckg in packages]
            for pckg in packages:
                if (pckg not in crossref_diff.keys()) and (f'{pckg} **pip**' not in crossref_diff.keys()):
                    raise PackageNotFound(f'{pckg} not found in either of the environments!')
        elif type(packages) == str:
            packages = packages.lower()
            if (packages not in crossref_diff.keys()) and (f'{packages} **pip**' not in crossref_diff.keys()):
                    raise PackageNotFound(f'{packages} not found in either of the environments!')
        else:
            raise TypeError(f'packages must be of type str or list!')

    p_max = 0
    v_max = max([len(env1_name), len(env2_name)])
    for p in crossref_diff:
        if len(p) > p_max:
            p_max = len(p)
        try:
            if len(env1[p]) > v_max:
                v_max = len(env1[p])
        except:
            continue
    p_max_spaces = ''.join([' ' for l in range(p_max+9)])
    v_max_spaces = ''.join([' ' for l in range(((v_max-len(env1_name))+15))])
    equal = True
    for ind, p in enumerate(crossref_diff):
        if crossref_diff[p] != 'both':
            equal = False
        else:
            if env1[p] != env2[p]:
                equal = False
    if equal:
        return print(f"\n{Fore.GREEN} No differences found between the two environments. Congratulations!{Style.RESET_ALL}\n")
    for ind, p in enumerate(crossref_diff):
        if ind == 0:
            print('\n')
            if compare_build:
                null = 'null            null'
                env1_name_start = str(f"{''.join([' ' for s in range(len(p_max_spaces))])}" + 
                                    f"{''.join([' ' for s in range(floor((v_max/2)-(ceil(2*(len(env1_name)/3)))))])}")
                env1_name_end = env1_name_start + ''.join([' ' for s in env1_name])
                env2_start = f"{''.join([' ' for s in range((len(p_max_spaces)+v_max+15))])}"
                env2_name_start = str(''.join([' ' for s in range(len(env2_start) - len(env1_name_end))]) + f"{''.join([' ' for s in range((floor(max([len(v) for v in env2.values()])/2)-len(env2_name)))])}")
                print(f"{env1_name_start}{Fore.CYAN}{env1_name}{Style.RESET_ALL}"
                      f"{env2_name_start}{Fore.CYAN}{env2_name}{Style.RESET_ALL}")
                print('')
                # 10 for 6 spaces used between first version and build (below) plus the length of 'build' (i.e. 5 characters) -1.
                print(f"{p_max_spaces}{Fore.BLUE}version{Style.RESET_ALL}"
                      f"{''.join([' ' for s in range(16-len('build'))])}"
                      f"{Fore.BLUE}build{Style.RESET_ALL}"
                      f"{''.join([' ' for s in range(len(env2_start)-len(p_max_spaces)-max([len(v.split(' ')[0]) for v in env1.values()])-len('verion'+'build'))])}"
                      f"{Fore.BLUE}version{Style.RESET_ALL}{''.join([' ' for s in range(max([len(v.split(' ')[0]) for v in env2.values()]))])}"
                      f"{Fore.BLUE}build{Style.RESET_ALL}")
            else:
                null = 'null'
                print(f'{p_max_spaces}{Fore.CYAN}{env1_name}{Style.RESET_ALL}{v_max_spaces}{Fore.CYAN}{env2_name}{Style.RESET_ALL}')
            print('')
        if crossref_diff[p] == 'both':
            pckg_spaces = ''.join([' ' for l in range((p_max+10)-len(p))])
            version_spaces = ''.join([' ' for l in range((v_max+15)-len(env1[p]))])
            if env1[p] == env2[p]:
                if diff_only:
                    continue
                if (packages==None) or ((type(packages)==str) and (p!=packages)) or ((type(packages)==list) and (not any(p == pckg for pckg in packages)) and (not any(p.split(' **pip**')[0] == pckg for pckg in packages))):
                    if '**pip**' not in p:
                        print(f'{p}{pckg_spaces}{Fore.GREEN}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.GREEN}{env2[p]}{Style.RESET_ALL}')
                    else:
                        print(f"{p.split(' **pip**')[0]}{Fore.RED} **pip**{Style.RESET_ALL}{pckg_spaces}"
                              f"{Fore.GREEN}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.GREEN}{env2[p]}{Style.RESET_ALL}")
                else:
                    if '**pip**' not in p:
                        print(f'{Fore.MAGENTA}{p}{Style.RESET_ALL}{pckg_spaces}{Fore.GREEN}{env1[p]}{Style.RESET_ALL}'
                              f'{version_spaces}{Fore.GREEN}{env2[p]}{Style.RESET_ALL}')
                    else:
                        print(f"{Fore.MAGENTA}{p.split(' **pip**')[0]}{Style.RESET_ALL}{Fore.RED} **pip**{Style.RESET_ALL}"
                              f"{pckg_spaces}{Fore.GREEN}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.GREEN}{env2[p]}{Style.RESET_ALL}")
            else:
                if (packages==None) or ((type(packages)==str) and (p!=packages)) or ((type(packages)==list) and (not any(p == pckg for pckg in packages)) and (not any(p.split(' **pip**')[0] == pckg for pckg in packages))):
                    if '**pip**' not in p:
                        print(f'{p}{pckg_spaces}{Fore.YELLOW}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.YELLOW}{env2[p]}{Style.RESET_ALL}')
                    else:
                        print(f"{p.split(' **pip**')[0]}{Fore.RED} **pip**{Style.RESET_ALL}{pckg_spaces}"
                              f"{Fore.YELLOW}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.YELLOW}{env2[p]}{Style.RESET_ALL}")
                else:
                    if '**pip**' not in p:
                        print(f'{Fore.MAGENTA}{p}{Style.RESET_ALL}{pckg_spaces}{Fore.YELLOW}{env1[p]}{Style.RESET_ALL}'
                              f'{version_spaces}{Fore.YELLOW}{env2[p]}{Style.RESET_ALL}')
                    else:
                        print(f"{Fore.MAGENTA}{p.split(' **pip**')[0]}{Style.RESET_ALL}{Fore.RED} **pip**{Style.RESET_ALL}"
                              f"{pckg_spaces}{Fore.YELLOW}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.YELLOW}{env2[p]}{Style.RESET_ALL}")
            header = False
        elif crossref_diff[p] == 'env1':
            pckg_spaces = ''.join([' ' for l in range((p_max+10)-len(p))])
            version_spaces = ''.join([' ' for l in range((v_max+15)-len(env1[p]))])
            if (packages==None) or ((type(packages)==str) and (p!=packages)) or ((type(packages)==list) and (not any(p == pckg for pckg in packages)) and (not any(p.split(' **pip**')[0] == pckg for pckg in packages))):
                if '**pip**' not in p:
                    print(f'{p}{pckg_spaces}{Fore.RED}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.RED}{null}{Style.RESET_ALL}')
                else:
                    print(f"{p.split(' **pip**')[0]}{Fore.RED} **pip**{Style.RESET_ALL}{pckg_spaces}{Fore.RED}{env1[p]}{Style.RESET_ALL}"
                          f"{version_spaces}{Fore.RED}{null}{Style.RESET_ALL}")
            else:
                if '**pip**' not in p:
                    print(f'{Fore.MAGENTA}{p}{Style.RESET_ALL}{pckg_spaces}{Fore.RED}{env1[p]}{Style.RESET_ALL}'
                          f'{version_spaces}{Fore.RED}{null}{Style.RESET_ALL}')
                else:
                    print(f"{Fore.MAGENTA}{p.split(' **pip**')[0]}{Style.RESET_ALL}{Fore.RED} **pip**{Style.RESET_ALL}"
                          f"{pckg_spaces}{Fore.RED}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.RED}{null}{Style.RESET_ALL}")
            header = False
        elif crossref_diff[p] == 'env2':
            pckg_spaces = ''.join([' ' for l in range((p_max+10)-len(p))])
            version_spaces = ''.join([' ' for l in range((v_max+15)-len(null))])
            if (packages==None) or ((type(packages)==str) and (p!=packages)) or ((type(packages)==list) and (not any(p == pckg for pckg in packages)) and (not any(p.split(' **pip**')[0] == pckg for pckg in packages))):
                if '**pip**' not in p:
                    print(f'{p}{pckg_spaces}{Fore.RED}{null}{Style.RESET_ALL}{version_spaces}{Fore.RED}{env2[p]}{Style.RESET_ALL}')
                else:
                    print(f"{p.split(' **pip**')[0]}{Fore.RED} **pip**{Style.RESET_ALL}{pckg_spaces}{Fore.RED}{null}{Style.RESET_ALL}"
                          f"{version_spaces}{Fore.RED}{env2[p]}{Style.RESET_ALL}")
            else:
                if '**pip**' not in p:
                    print(f'{Fore.MAGENTA}{p}{Style.RESET_ALL}{pckg_spaces}{Fore.RED}{null}{Style.RESET_ALL}'
                          f'{version_spaces}{Fore.RED}{env2[p]}{Style.RESET_ALL}')
                else:
                    print(f"{Fore.MAGENTA}{p.split(' **pip**')[0]}{Style.RESET_ALL}{Fore.RED} **pip**{Style.RESET_ALL}"
                          f"{pckg_spaces}{Fore.RED}{null}{Style.RESET_ALL}{version_spaces}{Fore.RED}{env2[p]}{Style.RESET_ALL}")
            header = False

In [7]:
[str(subprocess.run('bash -c "conda env list"', capture_output=True, shell=True)).split(',')[-2].replace(" ", "").split('\\')]

[["stdout=b'#condaenvironments:",
  'n#',
  'nFCA/home/jovyan/my-conda-envs/FCA',
  'nFCA_base/home/jovyan/my-conda-envs/FCA_base',
  'nFCA_test/home/jovyan/my-conda-envs/FCA_test',
  'nVCS/home/jovyan/my-conda-envs/VCS',
  'ncircos/home/jovyan/my-conda-envs/circos',
  'nenv_manage/home/jovyan/my-conda-envs/env_manage',
  'nfetal_skin/home/jovyan/my-conda-envs/fetal_skin',
  'nmetaflow/home/jovyan/my-conda-envs/metaflow',
  'nmilo/home/jovyan/my-conda-envs/milo',
  'nmonocle3/home/jovyan/my-conda-envs/monocle3',
  'nmonocle3_sceasy/home/jovyan/my-conda-envs/monocle3_sceasy',
  'nqc/home/jovyan/my-conda-envs/qc',
  'ntestenv/home/jovyan/my-conda-envs/testenv',
  'nbase*/opt/conda',
  'nr-reticulate/opt/conda/envs/r-reticulate',
  'n',
  "n'"]]

In [70]:
def build_sub_env(yml_file, packages, new_env_name=None, mamba =True, overide_env=False, prefix=None, constraint='exact', image=False, exception=None):
    """Takes in a yml file for the query env 
    and creates a new environment with additional
    specified packages.
    ----------------------------------------------------------
    Parameters:
    
    """
    
    import subprocess
    import time
    
    assert type(yml_file) == str, "'yml_file' must be a str"
    assert (type(packages) == list) or (type(packages) == dict), "'packages' " + \
    "must be either a dict of package:versions or a list of package names."
    constraint_type = {'fuzzy' : '=', 'exact' : '==', 'GoE' : '>=', 
                       'greater' : '>', 'SoE' : '<=', 'smaller' : '<'}
    assert constraint in list(constraint_type.keys()), f"'constraint' must be set " + \
    "to one of the {list(constraint_type.keys())}. See the docstring for more info."
    if type(packages) == list:
        assert all(type(p) == str for p in packages), "All specified packages " + \
        "in packages must of type str!"
    elif type(packages) == dict:
        assert all(type(p) == str for p in packages.keys()), "All " + \
        "specified package names in packages must of type str!"
        assert all(type(v) == str for v in packages.values()), "All \
        specified package versions in packages must of type str!"
        assert all(any(v.startswith(c) for c in list(constraint_type.values())) \
                   for v in packages.values() if v!=''), "All specified versions " + \
                   "must have constraint specifications!"
    
    
    ext_name = yml_file.split('/')[-1].split('.')[0]
    all_envs = str(subprocess.run('bash -c "conda env list"', 
                                  capture_output=True, 
                                  shell=True))
    if not new_env_name:
        print(f"No explicit name provided for the new environment. Falling back " 
              f"to the .yml file's name: '{ext_name}'.")
        new_env_name = ext_name
    if new_env_name in all_envs:
        assert overide_env==True, f"{new_env_name} env already exists! If you " + \
        "wish to overwrite this env, you must set 'overide_env' to True."
        print(f"Removing the exisitng {new_env_name}, you have 10 seconds to halt"
              "this process!")
        time.sleep(10)
        subprocess.run(f"bash -c 'conda env remove -n {new_env_name} -y'", 
                       shell=True)
    
    env, channels = load_env(yml_file)
    env = {p:env['version'][i] for i, p in enumerate(env['package'])}
    channels = [f'-c {ch}' for ch in channels]
    command = []
    if mamba:
        command.append(f'mamba create -n {new_env_name} -y')
    else:
        command.append(f'conda create -n {new_env_name} -y')
    command.extend(channels)
    
    if image:
        command.extend([f'{constraint_type[constraint]}'.join([k.split(' **pip**')[0], v]) for k,v in env.items()])
    
    if type(packages)==dict:
        for pack in packages:
            if pack in exception:
                continue
            for s in constraint_type.values():
                pack = pack.split(s)[-1]
            if any(pack in p for p in command):
                continue
            if packages[pack] == '':
                try:
                    command.append(f'{pack}{constraint_type[constraint]}{env[pack]}')
                except KeyError:
                    command.append(pack)
            else:
                command.append(f'{pack}{packages[pack]}')
    else:
        for pack in packages:
            if pack in exception:
                continue
            for s in constraint_type.values():
                pack = pack.split(s)[-1]
            if any(pack in p for p in command):
                continue
            try:
                command.append(f'{pack}{constraint_type[constraint]}{env[pack]}')
            except KeyError:
                    command.append(pack)
            
    command = ' '.join(command)
    print(command)
#     subprocess.run(f"bash -c '{command}'", 
#                    shell=True)
    

In [ ]:
def rebuild_env(yml_file, packages, mamba=True):
    """"""
    import johnnydep
    
    

In [24]:
import johnnydep

johnnydep.JohnnyDist('johnnydep').requires

2022-11-22 12:34.40 [info     ] init johnnydist                dist=johnnydep parent=None
2022-11-22 12:34.40 [debug    ] fetching best wheel            dist=johnnydep


['anytree',
 'cachetools',
 'colorama',
 'oyaml',
 'packaging>=17',
 'pip',
 'pkginfo>=1.4.2',
 'setuptools>=38.3',
 'structlog',
 'tabulate',
 'toml',
 'wheel>=0.32.0',
 'wimpy']

In [14]:
import johnnydep as pd
pd.__version__

'1.15'

In [171]:
subprocess.run('bash -c "johnnydep numpy"', shell=True)

bash: johnnydep: command not found


CompletedProcess(args='bash -c "johnnydep numpy"', returncode=127)

In [28]:
import subprocess
'FCA_base' in str(subprocess.run('bash -c "conda env list"', capture_output=True, shell=True))

True

In [38]:
# fa2 is required, because sctk.fdg (_fdg.py) does not currently does not 
# currently allow "fr" layout, returned by scanpy.tl.draw_graph.
init_package_ls = {'numpy', 'scanpy', 'anndata', 'sctk', 'cellrank', 'scvelo', 
              'seaborn', 'matplotlib', 'pandas', 'numpy_groupies', 
              'scikit-learn', 'plotnine', 'fa2', 'scirpy', 'harmonypy', 'umap-learn'}
package_versions = dict()
channels = []
env, channels = load_env(str(Path(root, 'logs', 'FCA_init.yml').resolve()))
for p in init_package_ls:
    if not any(p==pkg for pkg in env['package']):
        print(f'{p} not found in the .yaml file')
    else:
        package_versions[p] = env['version'][env['package'].index(p)]

sctk not found in the .yaml file
scirpy not found in the .yaml file


In [39]:
channels

['conda-forge', 'bioconda', 'defaults']

- plotnine can be installed with ```conda install -c conda-forge plotnine```

- SCTK can currently only be installed by```pip install``` exclusively.

- SCTK was cloned from git by ```git clone git@github.com:Teichlab/sctk.git```.

- plotnine was added to ```package_versions``` without a version (channel: conda-forge)

In [40]:
package_versions

{'cellrank': '1.0.0',
 'pandas': '1.1.2',
 'seaborn': '0.10.1',
 'scikit-learn': '1.0.2',
 'harmonypy': '0.0.5',
 'matplotlib': '3.3.2',
 'anndata': '0.7.5',
 'scvelo': '0.2.2',
 'plotnine': '0.7.1',
 'fa2': '0.3.5',
 'scanpy': '1.7.0',
 'umap-learn': '0.4.6',
 'numpy_groupies': '0.9.13',
 'numpy': '1.19.1'}

<span style='color:salmon'> The environment with these specified dependencies was causing issues with reading of  ```.h5ad``` files; converting the strings, namely the column names, into bytes, i.e. ```b"string"``` format. <br /> This is a know issue caused by ```h5py>2.10.0``` mixed with ```anndata=0.7.x```. To fix, ```h5py=2.10.0``` is manually added to the ```package_versions``` (and ```fussy_deps```) and to [follow tested solutions](https://github.com/scverse/anndata/issues/442) ```~/FCA_Fetal_Skin_priv/logs/FCA_init.yml``` was modified by changing ```Anndata=0.7.4``` to ```Anndata=0.7.5``` <b>before running the following</b>. </span>

<span style='color:salmon'> Furthermore, as ```SCTK``` (and subsequently its dependencies) must be installed exclusively through ```pip```, measures are taken to avoid the conflicts often caused by using ```pip``` in and environment managed by ```conda```.</span>:

- [X] Dependencies which are already installed in the conda env are isolated (```adjustText=0.7.3, igraph=0.10.2, leidenalg=0.9.0, texttable-1.6.4```), manually added (see below) to ```package_versions``` (and ```fussy_deps```) and consequently installed via conda. 
- [X] ```SCTK``` is ```git clone```ed and installed via ```.../sctk$ pip install --no-deps .```.

In [41]:
package_versions['h5py']='==2.10.0'
package_versions['adjustText']='==0.7.3'
package_versions['igraph']='==0.10.2'
package_versions['leidenalg']='==0.9.0'
package_versions['texttable']='==1.6.4'
package_versions['scirpy']=''
package_versions['umap-learn']='==0.5.1' # Most up-to-date version, which doesn't break the umap manifold. Ignoring, the one found in FCA_init.yml
for k,v in package_versions.items():
    if ('==' not in v) and (v != ''):
        package_versions[k] = '=='+v
package_versions

{'cellrank': '==1.0.0',
 'pandas': '==1.1.2',
 'seaborn': '==0.10.1',
 'scikit-learn': '==1.0.2',
 'harmonypy': '==0.0.5',
 'matplotlib': '==3.3.2',
 'anndata': '==0.7.5',
 'scvelo': '==0.2.2',
 'plotnine': '==0.7.1',
 'fa2': '==0.3.5',
 'scanpy': '==1.7.0',
 'umap-learn': '==0.5.1',
 'numpy_groupies': '==0.9.13',
 'numpy': '==1.19.1',
 'h5py': '==2.10.0',
 'adjustText': '==0.7.3',
 'igraph': '==0.10.2',
 'leidenalg': '==0.9.0',
 'texttable': '==1.6.4',
 'scirpy': ''}

In [131]:
subprocess.run(["conda", "", "/dev/null"], capture_output=True)

FileNotFoundError: [Errno 2] No such file or directory: 'lsd'

In [71]:
build_sub_env(str(Path(root, 'logs', 'monocle3.yml').resolve()), 
              packages=['r-sceasy'], 
              new_env_name='monocle3_sceasy', 
              constraint='fuzzy', 
              image=True, exception=['sctk'])

mamba create -n monocle3_sceasy -y -c conda-forge -c bioconda -c defaults _libgcc_mutex=0.1 _openmp_mutex=4.5 _r-mutex=1.0.1 anndata=0.7.8 binutils_impl_linux-64=2.36.1 binutils_linux-64=2.36 bioconductor-batchelor=1.8.0 bioconductor-beachmat=2.8.0 bioconductor-biobase=2.52.0 bioconductor-biocgenerics=0.38.0 bioconductor-biocneighbors=1.10.0 bioconductor-biocparallel=1.26.0 bioconductor-biocsingular=1.8.0 bioconductor-delayedarray=0.18.0 bioconductor-delayedmatrixstats=1.14.0 bioconductor-genomeinfodb=1.28.0 bioconductor-genomeinfodbdata=1.2.6 bioconductor-genomicranges=1.44.0 bioconductor-iranges=2.26.0 bioconductor-limma=3.48.0 bioconductor-matrixgenerics=1.4.0 bioconductor-residualmatrix=1.2.0 bioconductor-s4vectors=0.30.0 bioconductor-scaledmatrix=1.0.0 bioconductor-scuttle=1.2.0 bioconductor-singlecellexperiment=1.14.1 bioconductor-sparsematrixstats=1.4.0 bioconductor-summarizedexperiment=1.22.0 bioconductor-xvector=0.32.0 bioconductor-zlibbioc=1.38.0 boost-cpp=1.74.0 bwidget=1.9.

In [ ]:
mamba create -n FCA_R -y -c conda-forge -c bioconda -c defaults _libgcc_mutex=0.1 _openmp_mutex=4.5 _r-mutex=1.0.1 anndata=0.7.8 binutils_impl_linux-64=2.36.1 binutils_linux-64=2.36 bioconductor-batchelor=1.8.0 bioconductor-beachmat=2.8.0 bioconductor-biobase=2.52.0 bioconductor-biocgenerics=0.38.0 bioconductor-biocneighbors=1.10.0 bioconductor-biocparallel=1.26.0 bioconductor-biocsingular=1.8.0 bioconductor-delayedarray=0.18.0 bioconductor-delayedmatrixstats=1.14.0 bioconductor-genomeinfodb=1.28.0 bioconductor-genomeinfodbdata=1.2.6 bioconductor-genomicranges=1.44.0 bioconductor-iranges=2.26.0 bioconductor-limma=3.48.0 bioconductor-matrixgenerics=1.4.0 bioconductor-residualmatrix=1.2.0 bioconductor-s4vectors=0.30.0 bioconductor-scaledmatrix=1.0.0 bioconductor-scuttle=1.2.0 bioconductor-singlecellexperiment=1.14.1 bioconductor-sparsematrixstats=1.4.0 bioconductor-summarizedexperiment=1.22.0 bioconductor-xvector=0.32.0 bioconductor-zlibbioc=1.38.0 boost-cpp=1.74.0 bwidget=1.9.14 bzip2=1.0.8 c-ares=1.18.1 ca-certificates=2022.5.18.1 cached-property=1.5.2 cached_property=1.5.2 cairo=1.16.0 cfitsio=3.470 curl=7.79.1 expat=2.4.1 font-ttf-dejavu-sans-mono=2.37 font-ttf-inconsolata=3.000 font-ttf-source-code-pro=2.038 font-ttf-ubuntu=0.83 fontconfig=2.13.1 fonts-conda-ecosystem=1 fonts-conda-forge=1 freetype=2.10.4 freexl=1.0.6 fribidi=1.0.10 gcc_impl_linux-64=9.4.0 gcc_linux-64=9.4.0 geos=3.9.1 geotiff=1.7.0 gettext=0.19.8.1 gfortran_impl_linux-64=9.4.0 gfortran_linux-64=9.4.0 giflib=5.2.1 gmp=6.2.1 graphite2=1.3.13 gsl=2.7 gxx_impl_linux-64=9.4.0 gxx_linux-64=9.4.0 h5py=3.4.0 harfbuzz=3.0.0 hdf4=4.2.15 hdf5=1.12.1 icu=68.2 jbig=2.1 jpeg=9d json-c=0.15 kealib=1.4.14 kernel-headers_linux-64=2.6.32 krb5=1.19.2 lcms2=2.12 ld_impl_linux-64=2.36.1 lerc=3.0 libblas=3.9.0 libcblas=3.9.0 libcurl=7.79.1 libdap4=3.20.6 libdeflate=1.8 libedit=3.1.20191231 libev=4.33 libffi=3.4.2 libgcc-devel_linux-64=9.4.0 libgcc-ng=11.2.0 libgdal=3.3.2 libgfortran-ng=11.2.0 libgfortran5=11.2.0 libglib=2.70.0 libgomp=11.2.0 libiconv=1.16 libkml=1.3.0 liblapack=3.9.0 libnetcdf=4.8.1 libnghttp2=1.43.0 libnsl=2.0.0 libopenblas=0.3.18 libpng=1.6.37 libpq=13.3 librttopo=1.1.0 libsanitizer=9.4.0 libspatialite=5.0.1 libssh2=1.10.0 libstdcxx-devel_linux-64=9.4.0 libstdcxx-ng=11.2.0 libtiff=4.3.0 libuuid=2.32.1 libwebp-base=1.2.1 libxcb=1.13 libxml2=2.9.12 libzip=1.8.0 libzlib=1.2.11 lz4-c=1.9.3 make=4.3 natsort=8.0.0 ncurses=6.2 nspr=4.30 nss=3.69 numpy=1.21.4 openjpeg=2.4.0 openssl=1.1.1o packaging=21.0 pandas=1.3.4 pandoc=2.15 pango=1.48.10 pcre=8.45 pcre2=10.37 pip=21.3.1 pixman=0.40.0 poppler=21.09.0 poppler-data=0.4.11 postgresql=13.3 proj=8.1.1 pthread-stubs=0.4 pyparsing=3.0.6 python=3.9.7 python-dateutil=2.8.2 python_abi=3.9 pytz=2021.3 r-askpass=1.1 r-assertthat=0.2.1 r-backports=1.3.0 r-base=4.1.1 r-base64enc=0.1_3 r-bh=1.75.0_0 r-bit=4.0.4 r-bit64=4.0.5 r-bitops=1.0_7 r-blob=1.2.2 r-boot=1.3_28 r-brio=1.1.2 r-broom=0.7.9 r-bslib=0.3.1 r-cachem=1.0.6 r-callr=3.7.0 r-catools=1.18.2 r-cellranger=1.1.0 r-class=7.3_19 r-classint=0.4_3 r-cli=3.1.0 r-clipr=0.7.1 r-cluster=2.1.2 r-coda=0.19_4 r-codetools=0.2_18 r-colorspace=2.0_2 r-commonmark=1.7 r-cpp11=0.4.0 r-crayon=1.4.1 r-crosstalk=1.1.1 r-curl=4.3.2 r-cyclocomp=1.1.0 r-data.table=1.14.2 r-dbi=1.1.1 r-dbplyr=2.1.1 r-deldir=1.0_6 r-dendextend=1.15.2 r-desc=1.4.0 r-diffobj=0.3.5 r-digest=0.6.28 r-dplyr=1.0.7 r-dqrng=0.3.0 r-dtplyr=1.1.0 r-e1071=1.7_9 r-ellipsis=0.3.2 r-evaluate=0.14 r-expm=0.999_6 r-fansi=0.4.2 r-farver=2.1.0 r-fastmap=1.1.0 r-fnn=1.1.3 r-fontawesome=0.2.2 r-forcats=0.5.1 r-foreach=1.5.2 r-formatr=1.11 r-fs=1.5.0 r-furrr=0.2.3 r-futile.logger=1.4.3 r-futile.options=1.0.1 r-future=1.22.1 r-gargle=1.2.0 r-gclus=1.3.2 r-gdata=2.18.0 r-generics=0.1.1 r-ggnewscale=0.4.7 r-ggplot2=3.3.5 r-ggrepel=0.9.1 r-globals=0.14.0 r-glue=1.4.2 r-gmodels=2.18.1 r-googledrive=2.0.0 r-googlesheets4=1.0.0 r-gplots=3.1.1 r-gridextra=2.3 r-grr=0.9.5 r-gtable=0.3.0 r-gtools=3.9.2 r-haven=2.4.3 r-here=1.0.1 r-hexbin=1.28.2 r-highr=0.9 r-hms=1.1.1 r-htmltools=0.5.2 r-htmlwidgets=1.5.4 r-httpuv=1.6.3 r-httr=1.4.2 r-hunspell=3.0.1 r-ids=1.0.1 r-igraph=1.2.7 r-irlba=2.3.3 r-isoband=0.2.5 r-iterators=1.0.14 r-jquerylib=0.1.4 r-jsonlite=1.7.2 r-kernsmooth=2.23_20 r-knitr=1.35 r-labeling=0.4.2 r-lambda.r=1.2.4 r-later=1.2.0 r-lattice=0.20_45 r-lazyeval=0.2.2 r-learnbayes=2.15.1 r-leidenbase=0.1.3 r-lifecycle=1.0.1 r-lintr=2.0.1 r-listenv=0.8.0 r-lmtest=0.9_38 r-lobstr=1.1.1 r-lubridate=1.8.0 r-magrittr=2.0.1 r-markdown=1.1 r-mass=7.3_54 r-matrix=1.3_4 r-matrix.utils=0.9.8 r-matrixstats=0.61.0 r-mgcv=1.8_38 r-mime=0.12 r-modeldata=0.1.1 r-modelr=0.1.8 r-monocle3=1.0.0 r-munsell=0.5.0 r-nlme=3.1_153 r-openssl=1.4.5 r-paralleldist=0.2.6 r-parallelly=1.28.1 r-pbapply=1.5_0 r-pbmcapply=1.5.0 r-pheatmap=1.0.12 r-pillar=1.6.4 r-pkgconfig=2.0.3 r-pkgload=1.2.3 r-plotly=4.10.0 r-plyr=1.8.6 r-png=0.1_7 r-polychrome=1.3.1 r-praise=1.0.0 r-prettyunits=1.1.1 r-processx=3.5.2 r-progress=1.2.2 r-promises=1.2.0.1 r-proxy=0.4_26 r-pryr=0.1.5 r-ps=1.6.0 r-pscl=1.5.5 r-purrr=0.3.4 r-qap=0.1_1 r-r.cache=0.15.0 r-r.methodss3=1.8.1 r-r.oo=1.24.0 r-r.utils=2.11.0 r-r6=2.5.1 r-rann=2.6.1 r-rappdirs=0.3.3 r-raster=3.5_2 r-rcolorbrewer=1.1_2 r-rcpp=1.0.7 r-rcppannoy=0.0.19 r-rcpparmadillo=0.10.8.1.0 r-rcppeigen=0.3.3.9.1 r-rcpphnsw=0.3.0 r-rcppparallel=5.1.4 r-rcppprogress=0.4.2 r-rcurl=1.98_1.5 r-readr=2.0.2 r-readxl=1.3.1 r-registry=0.5_1 r-rematch=1.0.1 r-rematch2=2.1.2 r-remotes=2.4.2 r-reprex=2.0.1 r-reshape2=1.4.4 r-reticulate=1.22 r-rex=1.2.1 r-rhpcblasctl=0.21_247 r-rlang=0.4.12 r-rmarkdown=2.11 r-rprojroot=2.0.2 r-rsample=0.1.0 r-rspectra=0.16_0 r-rstudioapi=0.13 r-rsvd=1.0.5 r-rtsne=0.15 r-rvest=1.0.2 r-s2=1.0.7 r-sass=0.4.0 r-scales=1.1.1 r-scatterplot3d=0.3_41 r-selectr=0.4_2 r-seriation=1.3.2 r-sf=1.0_3 r-shiny=1.7.1 r-sitmo=2.0.2 r-slam=0.1_48 r-slider=0.2.2 r-snow=0.4_4 r-sourcetools=0.1.7 r-sp=1.4_5 r-spdata=2.0.1 r-spdep=1.1_11 r-speedglm=0.3_3 r-spelling=2.2 r-stringdist=0.9.8 r-stringi=1.7.5 r-stringr=1.4.0 r-styler=1.6.2 r-sys=3.4 r-terra=1.4_11 r-testthat=3.1.0 r-tibble=3.1.5 r-tidyr=1.1.4 r-tidyselect=1.1.1 r-tidyverse=1.3.1 r-tinytex=0.34 r-tsp=1.2_0 r-tzdb=0.2.0 r-units=0.7_2 r-utf8=1.2.2 r-uuid=1.0_2 r-uwot=0.1.10 r-vctrs=0.3.8 r-viridis=0.6.2 r-viridislite=0.4.0 r-vroom=1.5.5 r-waldo=0.3.1 r-warp=0.2.0 r-withr=2.4.2 r-wk=0.5.0 r-xfun=0.27 r-xml2=1.3.2 r-xmlparsedata=1.0.5 r-xtable=1.8_4 r-yaml=2.2.1 r-zoo=1.8_9 readline=8.1 scipy=1.7.2 sed=4.8 setuptools=59.1.1 six=1.16.0 sqlite=3.36.0 sysroot_linux-64=2.12 tiledb=2.3.4 tk=8.6.11 tktable=2.10 tzcode=2021e tzdata=2021e udunits2=2.2.27.27 wheel=0.37.0 xerces-c=3.2.3 xorg-kbproto=1.0.7 xorg-libice=1.0.10 xorg-libsm=1.2.3 xorg-libx11=1.7.2 xorg-libxau=1.0.9 xorg-libxdmcp=1.1.3 xorg-libxext=1.3.4 xorg-libxrender=0.9.10 xorg-libxt=1.2.1 xorg-renderproto=0.11.1 xorg-xextproto=7.3.0 xorg-xproto=7.0.31 xz=5.2.5 zlib=1.2.11 zstd=1.5.0 r-sceasy conda-forge::r-seurat=4.1.0 r-recommended r-irkernel Jupyter

In [155]:
fussy_deps = ['anndata', 'scanpy', 'h5py',
              'adjustText','igraph','leidenalg',
              'texttable', 'umap-learn', 'harmonypy'] # list of version-specific packages

In [10]:
with Path(root, 'logs', 'FCA_base.sh').open(mode='w') as conda_install:
    conda_install.write(f'mamba create -n FCA_base')
    conda_install.write(" ")
    for ch in channels:
        conda_install.write(f'-c {ch}')
        conda_install.write(" ")
    for pack in package_versions:
        if pack in fussy_deps:
            conda_install.write(f'{pack}={package_versions[pack]}')
            conda_install.write(" ")
        else:
            conda_install.write(f'{pack}')
            conda_install.write(" ")

In [11]:
with Path(root, 'logs', 'FCA_base.sh').open(mode='r') as conda_install:
    for line in conda_install:
        print(line)

mamba create -n FCA_base -c conda-forge -c bioconda -c defaults -c conda-forge -c bioconda -c defaults anndata=0.7.5 cellrank fa2 harmonypy=0.0.5 matplotlib numpy numpy_groupies pandas plotnine scanpy=1.7.0 scikit-learn scvelo seaborn umap-learn=0.5.1 h5py=2.10.0 adjustText=0.7.3 igraph=0.10.2 leidenalg=0.9.0 texttable=1.6.4 scirpy 


In [12]:
# !chmod +x ~/FCA_Fetal_Skin_priv/logs/FCA_base.sh

In [13]:
# !~/FCA_Fetal_Skin_priv/logs/FCA_base.sh

In [14]:
# !conda activate FCA_base
# !cd ~/sctk
# !pip install . --no-deps
# !conda env export > ~/FCA_Fetal_Skin_priv/logs/FCA_base.yml

To cross-ref two ```conda envs```

In [8]:
import json

dependencies_file = json.load(open('../my-conda-envs/monocle3/conda-meta/r-monocle3-1.0.0-r41h7d875b9_1.json'))
dependencies = []
for k, v in dependencies_file.items():
    if 'depends' in k:
        for d in dependencies_file[k]:
            dependencies.append(d.split(' ')[0])

In [12]:
crossref_2_envs(str(Path(root, 'logs', 'monocle3_sceasy.yml').resolve()), 
                str(Path(root, 'logs', 'monocle3.yml').resolve()), 
                dependencies+['r-monocle3'], compare_build=False) # try compare_build=True if you think the build difference maybe important



                                          monocle3_sceasy               monocle3

_libgcc_mutex                              0.1                           0.1
_openmp_mutex                              4.5                           4.5
_r-mutex                                   1.0.1                         1.0.1
anndata                                    0.7.8                         0.7.8
asttokens                                  2.2.0                         null
backcall                                   0.2.0                         null
backports                                  1.1                           null
backports.functools_lru_cache              1.6.4                         null
binutils_impl_linux-64                     2.36.1                        2.36.1
binutils_linux-64                          2.36                          2.36
bioconductor-batchelor                     1.8.0                         1.8.0
bioconductor-beachmat                      2.8.0       

```sceasy``` not found in ```monocle3```!

Do the above process done for ```FCA_base``` for ```monocle3``` but this time add all the packages to the ```fussy_deps``` and then add ```r-sceasy``` without a specified version to the created ```monocle3.sh``` file. 

In [234]:
import json

import requests

package_name = 'tensorflow'
url = 'https://pypi.python.org/pypi/' + str(package_name) + '/json'
data = requests.get(url).json()

print(data['info']['requires_dist'])

['absl-py (>=1.0.0)', 'astunparse (>=1.6.0)', 'flatbuffers (>=2.0)', 'gast (<=0.4.0,>=0.2.1)', 'google-pasta (>=0.1.1)', 'h5py (>=2.9.0)', 'libclang (>=13.0.0)', 'numpy (>=1.20)', 'opt-einsum (>=2.3.2)', 'packaging', 'protobuf (<3.20,>=3.9.2)', 'setuptools', 'six (>=1.12.0)', 'termcolor (>=1.1.0)', 'typing-extensions (>=3.6.6)', 'wrapt (>=1.11.0)', 'grpcio (<2.0,>=1.24.3)', 'tensorboard (<2.12,>=2.11)', 'tensorflow-estimator (<2.12,>=2.11.0)', 'keras (<2.12,>=2.11.0)', 'tensorflow-io-gcs-filesystem (>=0.23.1) ; platform_machine != "arm64" or platform_system != "Darwin"']
